In [1]:
import cobra
import pandas as pd
from eflux.utils import get_flux_bounds, get_gpr_dict, gene_expression_to_enzyme_activity, convert_transcriptomics_to_enzyme_activity

In [2]:
def get_cobra_model():
    """Fixture for testing a toy cobra model."""
    model = cobra.Model("test_model")
    r1 = cobra.Reaction("r1")
    r2 = cobra.Reaction("r2")
    r3 = cobra.Reaction("r3")
    r4 = cobra.Reaction("r4")

    model.add_metabolites([cobra.Metabolite("m1"), cobra.Metabolite("m2"), cobra.Metabolite("m3")])
    r1.add_metabolites({model.metabolites.m1: 1})
    r2.add_metabolites({model.metabolites.m1: -1, model.metabolites.m2: 1})
    r3.add_metabolites({model.metabolites.m2: -1, model.metabolites.m3: 1})
    r4.add_metabolites({model.metabolites.m3: -1})

    # Set flux bounds for r1 and r2
    r2.bounds = (0.01, 10)
    r3.bounds = (0, 5)

    model.add_reactions([r1, r2, r3, r4])
    return model


def add_genes_to_r2(cobra_model):
    r2 = cobra_model.reactions.get_by_id("r2")
    r2.gene_reaction_rule = "gene1 and gene2"
    return cobra_model


def add_genes_to_r3(cobra_model_1):
    r3 = cobra_model_1.reactions.get_by_id("r3")
    r3.gene_reaction_rule = "(gene5 and gene6) or (gene7 and gene8)"
    return cobra_model_1



In [3]:
this_cobra_model = get_cobra_model()
this_cobra_model_1 = add_genes_to_r2(this_cobra_model)
this_cobra_model_2 = add_genes_to_r3(this_cobra_model_1)

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"


In [4]:
get_gpr_dict(this_cobra_model)

{<Reaction r2 at 0x177f0d190>: {frozenset({'gene1', 'gene2'})},
 <Reaction r3 at 0x104c7eed0>: {frozenset({'gene7', 'gene8'}),
  frozenset({'gene5', 'gene6'})}}

In [5]:
this_cobra_model.reactions.get_by_id('r2')

Reaction identifier,r2
Name,
Memory address,0x2b43ef3d0
Stoichiometry,m1 --> m2 -->
GPR,gene1 and gene2
Lower bound,0.01
Upper bound,10


In [6]:
this_cobra_model.reactions.get_by_id('r2').gene_reaction_rule = "(gene1 and gene2) or (gene3 and gene4)"
this_cobra_model.reactions.get_by_id('r2')

Reaction identifier,r2
Name,
Memory address,0x2b43ef3d0
Stoichiometry,m1 --> m2 -->
GPR,(gene1 and gene2) or (gene3 and gene4)
Lower bound,0.01
Upper bound,10


In [7]:
get_gpr_dict(this_cobra_model)

{<Reaction r2 at 0x2b43ef3d0>: {frozenset({'gene3', 'gene4'}),
  frozenset({'gene1', 'gene2'})},
 <Reaction r3 at 0x2b59e4850>: {frozenset({'gene5', 'gene6'}),
  frozenset({'gene7', 'gene8'})}}

In [8]:
rxns = list(this_cobra_model.reactions)
rxns

[<Reaction r1 at 0x111aa17d0>,
 <Reaction r2 at 0x2b43ef3d0>,
 <Reaction r3 at 0x2b59e4850>,
 <Reaction r4 at 0x2b59e5350>]

In [9]:
this_cobra_model_2

Name,test_model
Memory address,2b59e5490
Number of metabolites,3
Number of reactions,4
Number of genes,8
Number of groups,0
Objective expression,0
Compartments,


In [10]:
expression = {"gene1": 1.0, "gene2": 2.0, "gene3": 3.0, "gene4": 4.0, "gene5": 5.0, "gene6": 6.0, "gene7": 7.0, "gene8": 8.0}
expression

{'gene1': 1.0,
 'gene2': 2.0,
 'gene3': 3.0,
 'gene4': 4.0,
 'gene5': 5.0,
 'gene6': 6.0,
 'gene7': 7.0,
 'gene8': 8.0}

In [11]:
gpr = get_gpr_dict(this_cobra_model_2)
print(gpr)
gene_expression_to_enzyme_activity(this_cobra_model_2, gpr, expression)

{<Reaction r2 at 0x2b43ef3d0>: {frozenset({'gene4', 'gene3'}), frozenset({'gene2', 'gene1'})}, <Reaction r3 at 0x2b59e4850>: {frozenset({'gene5', 'gene6'}), frozenset({'gene7', 'gene8'})}}


{<Reaction r1 at 0x111aa17d0>: nan,
 <Reaction r2 at 0x2b43ef3d0>: 4.0,
 <Reaction r3 at 0x2b59e4850>: 12.0,
 <Reaction r4 at 0x2b59e5350>: nan}

In [12]:
transcriptomics = pd.DataFrame({'strain1': [1, 2, 3, 4, 5], 'strain2': [6, 7, 8, 9, 10]}, index=['gene1', 'gene2', 'gene3', 'gene5', 'gene6'])
transcriptomics

,strain1,strain2
gene1,1,6
gene2,2,7
gene3,3,8
gene5,4,9
gene6,5,10


In [14]:
convert_transcriptomics_to_enzyme_activity(transcriptomics, this_cobra_model_2)

,Reaction_ID,strain1,strain2
r1: --> m1,r1,NaN,NaN
r2: m1 --> m2,r2,4.0,14.0
r3: m2 --> m3,r3,inf,inf
r4: m3 -->,r4,NaN,NaN


In [22]:
gpr = get_gpr_dict(this_cobra_model_2)
print(gpr)
gene_expression_to_enzyme_activity(this_cobra_model_2, gpr, transcriptomics['strain2'])

{<Reaction r2 at 0x2b58ebf10>: {frozenset({'gene2', 'gene1'})}, <Reaction r3 at 0x2b706b690>: {frozenset({'gene6', 'gene5'}), frozenset({'gene7', 'gene8'})}}


{<Reaction r1 at 0x2b7662a50>: nan,
 <Reaction r2 at 0x2b58ebf10>: 6.0,
 <Reaction r3 at 0x2b706b690>: inf,
 <Reaction r4 at 0x2b7009190>: nan}

In [15]:
model = get_cobra_model()
model

Name,test_model
Memory address,2b438f410
Number of metabolites,3
Number of reactions,4
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [16]:
for r in model.reactions:
    print(r, r.bounds)

r1:  --> m1 (0.0, 1000.0)
r2: m1 --> m2 (0.01, 10)
r3: m2 --> m3 (0, 5)
r4: m3 -->  (0.0, 1000.0)


In [17]:
model.reactions.r4.lower_bound = 6

In [18]:
model.optimize(raise_error=True)

OptimizationError: Solver status is 'infeasible'.

In [19]:
dir(model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_annotation',
 '_compartments',
 '_contexts',
 '_id',
 '_populate_solver',
 '_repr_html_',
 '_set_id_with_model',
 '_solver',
 '_tolerance',
 'add_boundary',
 'add_cons_vars',
 'add_groups',
 'add_metabolites',
 'add_reactions',
 'annotation',
 'boundary',
 'compartments',
 'constraints',
 'copy',
 'demands',
 'exchanges',
 'genes',
 'get_associated_groups',
 'groups',
 'id',
 'medium',
 'merge',
 'metabolites',
 'name',
 'notes',
 'objective',
 'objective_direction',
 'optimize',
 'problem',
 'reactions',
 'remove_cons_vars',
 'remove_groups',
 'remove_metabol

In [26]:
for c in model.variables:
    print(c)

0 <= r1 <= 1000.0
0 <= r1_reverse_7c92c <= -0.0
0.01 <= r2 <= 10
0 <= r2_reverse_d2791 <= 0
0 <= r3 <= 5
0 <= r3_reverse_9d3e6 <= 0
6 <= r4 <= 1000.0
0 <= r4_reverse_44379 <= 0


In [25]:
for c in model._solver.variables:
    print(c)

0 <= r1 <= 1000.0
0 <= r1_reverse_7c92c <= -0.0
0.01 <= r2 <= 10
0 <= r2_reverse_d2791 <= 0
0 <= r3 <= 5
0 <= r3_reverse_9d3e6 <= 0
6 <= r4 <= 1000.0
0 <= r4_reverse_44379 <= 0


In [27]:
for c in model.problem.constraints:
    print(c)

AttributeError: module 'optlang.gurobi_interface' has no attribute 'constraints'

In [31]:
print([v for v in model.variables])



[0 <= r1 <= 1000.0, 0 <= r1_reverse_7c92c <= -0.0, 0.01 <= r2 <= 10, 0 <= r2_reverse_d2791 <= 0, 0 <= r3 <= 5, 0 <= r3_reverse_9d3e6 <= 0, 6 <= r4 <= 1000.0, 0 <= r4_reverse_44379 <= 0]


In [ ]:
# reaction_flux = model.problem.Variable('v', lb=0, ub=10)
r4 = model.reactions.r4

# Set the objective to maximize r4
model.objective = r4

# Optimize for growth to find the maximum r4 rate
r4_solution = model.optimize()
max_r4 = r4_solution.objective_value


slack_var = model.problem.Variable("r4_slack", lb=0)

# Add a slack variable via model.problem.Constraint
# slack_variable = model.problem.Variable('s', lb=0)
lower_bound_constraint = model.problem.Constraint(r4 + slack_variable, lb=6, name='lower_bound_constraint')
model.solver.add(lower_bound_constraint)

model.problem.


In [8]:
model = get_cobra_model()

model = add_genes_to_r2(model)
model = add_genes_to_r3(model)

rids = [r.id for r in model.reactions]
rids



opt_model, flux_bounds = get_flux_bounds(model, ['r1','r4'])
flux_bounds

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmp66gqrc10.lp
Reading time = 0.00 seconds
: 4 rows, 9 columns, 13 nonzeros
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmpdobpwsot.lp
Reading time = 0.00 seconds
: 4 rows, 9 columns, 13 nonzeros
Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmpr1sd9o96.lp
Reading time = 0.00 seconds
: 4 rows, 9 columns, 13 nonzeros
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmpohlnyad4.lp
Reading time = 0.00 seconds
: 4 rows, 9 columns, 13 nonzeros


,minimum,maximum
r2,0.01,5.0
r3,0.01,5.0


In [17]:

transcriptomics = pd.DataFrame({'strain1': [1, 2, 3, 4, 5], 'strain2': [6, 7, 8, 9, 10]}, index=['gene1', 'gene2', 'gene3', 'gene5', 'gene6'])

enzyme_activity = convert_transcriptomics_to_enzyme_activity(transcriptomics, model)

scaled_enzyme_activity = enzyme_activity
scaled_enzyme_activity = scaled_enzyme_activity.divide(scaled_enzyme_activity['strain2'], axis=0)
scaled_enzyme_activity


,strain1,strain2
Reaction_ID,,
r1,NaN,NaN
r2,0.166667,1.0
r3,NaN,NaN
r4,NaN,NaN


In [36]:
model = get_cobra_model()
model.reactions.r2.flux_expression


1.0*r2 - 1.0*r2_reverse_d2791

In [37]:
import cobra

# Load your metabolic model (this could be from a file or predefined model)
# model = cobra.io.load_model('your_model_file.json')
model = get_cobra_model()

# Define the growth objective as the biomass reaction
biomass_reaction = model.reactions.get_by_id('r4')

# Set the objective to maximize growth
model.objective = biomass_reaction

# # Optimize for growth to find the maximum growth rate
# growth_solution = model.optimize()
# max_growth = growth_solution.objective_value

# Add slack variables as additional reactions with only an upper bound
for reaction in model.reactions:
    # Create a new reaction representing the slack variable
    slack_reaction = cobra.Reaction('SLACK_' + reaction.id)
    slack_reaction.bounds = (0, 1000)  # Assuming a large upper bound for the slack variable
    # Add the slack reaction to the model
    model.add_reactions([slack_reaction])
    upper_bound = flux_bounds.loc[reaction.id, 'maximum'] * scaled_enzyme_activity[reaction.id]
    constraint = model.problem.Constraint(reaction.flux_expression - slack_reaction.flux_expression, lb=0, ub=upper_bound)
    
    # Link the slack reaction to the original reaction by modifying the bounds
    reaction.upper_bound += slack_reaction.flux_expression

# Define a new combined objective: maximize growth and minimize sum of slack variables
# Here we assume that the slack reactions have been named with a 'SLACK_' prefix
slack_reactions = [model.reactions.get_by_id('SLACK_' + r.id) for r in model.reactions]
combined_objective = model.problem.Objective(
    biomass_reaction.flux_expression - 0.001 * sum([r.flux_expression for r in slack_reactions]),
    direction='max'
)

# Set the combined objective as the objective of the model
model.objective = combined_objective

# Optimize the model with the combined objective
combined_solution = model.optimize()

AttributeError: property 'flux_expression' of 'Reaction' object has no setter

In [ ]:
new = model.problem.Constraint(model.objective.expression, lb=0.99)
model.solver.add(new)

In [ ]:
from optlang import Model, Variable, Constraint

# Create a model
model = Model(name='FBA_Model')

# Define a reaction variable (v) with a lower bound (lb) and upper bound (ub)
reaction_flux = Variable('v', lb=0, ub=10)

# Define a slack variable (s) for the upper bound, which must be non-negative
slack_variable = Variable('s', lb=0)

# Modify the upper bound constraint to include the slack variable
# The original upper bound is now the sum of the reaction flux and the slack variable
upper_bound_constraint = Constraint(reaction_flux - slack_variable, ub=10, name='upper_bound_constraint')

# Add the reaction flux and the slack variable to the model17model.add([reaction_flux, slack_variable])
# Add the modified upper bound constraint to the model
model.add(upper_bound_constraint)

# Now you can proceed with the optimization of the model
model.optimize()

In [ ]:
from optlang import Model, Variable, Constraint, Objective
# Assuming you have a model with reaction fluxes and slack variables
model = Model(name='FBA_Model')

# Define your reaction flux variable (e.g., biomass production reaction)
biomass_reaction = Variable('biomass', lb=0, ub=1000)

# Define slack variables for the upper bounds of other reactions
slack_variables = [Variable(f'slack_{i}', lb=0) for i in range(1, 4)]

# Add variables to the model
model.add([biomass_reaction] + slack_variables)

# Define constraints (not shown here) and add them to the model
# ...

# Now, set up the combined objective
# For example, you might want to maximize growth (biomass_reaction) while minimizing the sum of slack variables

# First, define the objective for maximizing growth
growth_objective = Objective(biomass_reaction, direction='max')

# Then, define the objective for minimizing the sum of slack variables
slack_objective = Objective(sum(slack_variables), direction='min')

# Combine the objectives into a single objective
# You can use weights to prioritize one objective over the other
# For example, you might want to prioritize growth over minimizing slack
weight_for_growth = 1
weight_for_slack = 0.01  # A smaller weight since we want to prioritize growth

combined_objective = Objective(weight_for_growth * growth_objective.expression - weight_for_slack * slack_objective.expression, direction='max')

# Set the combined objective as the objective of the model
model.objective = combined_objective

# Now you can optimize the model with the combined objective
solution = model.optimize()